In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
import cv2
from torchvision import transforms
model.eval()

In [ ]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
pred =newmodel(input_tensor)

In [ ]:
pred.shape

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609_c.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
target =newmodel(input_tensor)

In [1]:
from envs.env_dataloader import create_dataloaders
import torch
train,test = create_dataloaders()

Encoding testing data ...


  0%|          | 0/4 [00:00<?, ?it/s]

finished...
Encoding training data ...


  0%|          | 0/36 [00:00<?, ?it/s]

finished...


In [2]:
f=train.dataset.encoded_source

In [3]:
from envs.new_edit_photo import PhotoEditor

editor = PhotoEditor()
parameters = torch.tensor([0.125, 0.125, 0.375, 0.125, 0., 0.0625, 0.9375, 0.375, 0.0625, 0., 0.125, 0.125])
parameters = torch.stack([parameters for i in range(64)])

In [8]:
for i,j,k,l in train:
    # print(i.dtype)
    enhanced = i.permute(0,2,3,1)/255.0
    # print(enhanced.dtype)
    enhanced = editor(enhanced,parameters)
    enhanced = enhanced.permute(0,3,1,2)
    enhanced =torch.flatten(enhanced,start_dim=1, end_dim=-1)

    target = torch.flatten(j,start_dim=1, end_dim=-1)/255.0
    # print(enhanced.dtype)
    rmse = enhanced-target
    rmse = torch.pow(rmse,2).mean(1)
    rmse = torch.sqrt(rmse)
    break

print(rmse)

tensor([0.2333, 0.0819, 0.0687, 0.1057, 0.0855, 0.1581, 0.0719, 0.1410, 0.0450,
        0.0915, 0.1638, 0.1651, 0.0452, 0.1827, 0.1230, 0.2867, 0.1699, 0.1221,
        0.0647, 0.0479, 0.1827, 0.0721, 0.1205, 0.0519, 0.0575, 0.1052, 0.2729,
        0.1082, 0.0707, 0.0706, 0.1088, 0.2451, 0.0952, 0.0589, 0.1312, 0.0568,
        0.2146, 0.2209, 0.0827, 0.0626, 0.2599, 0.2217, 0.0855, 0.0643, 0.1531,
        0.0929, 0.0710, 0.2606, 0.0947, 0.1202, 0.1174, 0.2247, 0.1169, 0.1119,
        0.1799, 0.0965, 0.0792, 0.1051, 0.0620, 0.0655, 0.1262, 0.0787, 0.2578,
        0.0484])
